In [ ]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf()
sc = SparkSession.builder.config(conf=conf).getOrCreate()
sc.version

In [ ]:
import pyspark
import pandas as pd
import mysql.connector as mariadb

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder \
                    .master("local[*]") \
                    .appName("MyApp") \
                    .config("spark.driver.memory", "10g") \
                    .getOrCreate()
#sqlcontext = SQLContext(spark)

dbURL = "jdbc:mariadb://localhost:3306"
properties = {'user': 'user', 'password': 'password'}
sql = """ 
    select * 
    from db.table
    limit 10
"""

# mysql+mysqlconnector://user:***********@**********:3306/amatdbcharset=utf8mb4&collation=utf8mb4_general_ci
conn = mariadb.connect(user='user', database='db', password='password', host="localhost", port=3306, collation='utf8mb4_general_ci')
cursor = conn.cursor()
query = sql

pdf = pd.read_sql(query, con=conn)
conn.close()

df = spark.createDataFrame(pdf)
df.show(10)
df.printSchema()


In [ ]:
import os
import pyspark

from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///F:/path/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar pyspark-shell'
path = "F:/path/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar"

spark = SparkSession.builder \
                    .master("local[*]") \
                    .appName("Connect to MySql") \
                    .config("spark.driver.memory", "10g") \
                    .config("spark.jars", path) \
                    .config("spark.driver.extraClassPath", path)\
                    .config("spark.executor.extraClassPath", path)\
                    .getOrCreate()

sql = "select * from sakila.address"
database = "sakila"
user = "root"
password = "secret"
server = "localhost"
port = 3307
jdbc_url = "jdbc:mysql://localhost:3307"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("driver", jdbc_driver) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .load()

df.show(5)

In [ ]:
import os
import pyspark

from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///F:/path/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar pyspark-shell'
path = "F:/path/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar"

spark = SparkSession.builder \
    .appName("Connect to MariaDB") \
    .master("local") \
    .config("spark.jars", path)\
    .config("spark.driver.extraClassPath", path)\
    .config("spark.executor.extraClassPath", path)\
    .getOrCreate()

sql = "select * from db.table"

table_name = "table"
database = "db"
user = "user"
password = "password"
server = "localhost"
port = 3306
jdbc_url ="jdbc:mysql://localhost:3306/db"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("driver", jdbc_driver) \
    .option("dbtable", table_name) \
    .option("user", user) \
    .option("password", password) \
    .load()

df.printSchema()
df.show(5)